# Spark ML Production

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

spark = SparkSession\
    .builder\
    .master("local[*]")\
    .appName("Spark ML Production")\
    .getOrCreate()

23/07/27 12:25:36 WARN Utils: Your hostname, Air.local resolves to a loopback address: 127.0.0.1; using 192.168.88.21 instead (on interface en5)
23/07/27 12:25:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/07/27 12:25:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/07/27 12:25:36 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## Загружаем данные

In [2]:
data = spark\
    .read\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .csv("../../data/BankChurners.csv")

## Загружаем модель

In [3]:
from pyspark.ml import PipelineModel

model = PipelineModel.load("../pipelineModel")

## Вычисляем

In [4]:
predicted = model.transform(data)

In [5]:
predicted.show(10)

23/07/27 12:25:43 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---------+-----------------+------------+------+---------------+---------------+--------------+---------------+-------------+--------------+------------------------+----------------------+---------------------+------------+-------------------+---------------+--------------------+---------------+--------------+-------------------+---------------------+----------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------+--------------+-----------------------+----------------------+-----------------------+---------------------+--------------------+-----------------------------+----------------------------+-----------------------------+---------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|CLIENTNUM|   Attriti

## Проверяем результат

In [6]:
tp = predicted.filter((f.col("Attrition_Flag") == "Attrited Customer") & (f.col("prediction") == 1)).count()
tn = predicted.filter((f.col("Attrition_Flag") == "Existing Customer") & (f.col("prediction") == 0)).count()
fp = predicted.filter((f.col("Attrition_Flag") == "Existing Customer") & (f.col("prediction") == 1)).count()
fn = predicted.filter((f.col("Attrition_Flag") == "Attrited Customer") & (f.col("prediction") == 0)).count()

print(f"Confusion Matrix:\n{tp:>4}\t{fp:>4}\n{fn:>4}\t{tn:>4}")

Confusion Matrix:
1221	1780
 406	6720


In [7]:
accuracy = (tp + tn) / (tp + tn + fp + fn)
precision = tp / (tp + fp)
recall = tp / (tp + fn)

print(f"Accuracy = {accuracy}")
print(f"Precision = {precision}")
print(f"Recall = {recall}")

Accuracy = 0.7841414041670781
Precision = 0.4068643785404865
Recall = 0.750460971112477
